In [1]:
import json

import torch
import pandas as pd

pd.set_option('display.max_colwidth', 160)
pd.set_option('display.max_rows', 60)

df = pd.read_csv('train.csv')
df = df[df.count_region > 0]
df.head()

,ID_img,count_region,region_shape
8,3444.jpg,1.0,"['{""cx"":2259,""cy"":391,""r"":64}']"
217,3653.JPG,1.0,"['{""cx"":2719,""cy"":2097,""r"":75}']"
254,3690.JPG,1.0,"['{""cx"":2914,""cy"":1693,""r"":90}']"
399,3835.JPG,2.0,"['{""cx"":2549,""cy"":2329,""r"":80}', '{""cx"":2437,""cy"":2321,""r"":74}']"
511,3947.JPG,1.0,"['{""cx"":3130,""cy"":1370,""r"":134}']"


In [2]:
TH  = 0.45 # порог детекции людей
DIV = 5    # коэффициент, уменьшающий область вокруг объектов, по умолчаниюю 4

model = torch.hub.load('ultralytics/yolov5', 'custom', 'yolov5/runs/train/omsk-320v5/weights/best.pt').eval()

Using cache found in /home/andyst/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-8 Python-3.8.13 torch-1.13.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11179MiB)

Fusing layers... 
YOLOv5l6 summary: 346 layers, 76118664 parameters, 0 gradients, 109.9 GFLOPs
Adding AutoShape... 


In [3]:
preds = []
for img in df.ID_img.values:
    with torch.no_grad():
        results = model(f'train/{img}', augment=True)
    a = results.pred[0].cpu().numpy()
    a = a[a[:, 4] > TH]
    res = list(zip((a[:, 2] + a[:, 0]) / 2, (a[:, 1] + a[:, 3]) / 2, ((a[:, 2] - a[:, 0]) + (a[:, 3] - a[:, 1])) / DIV))
    res = sorted(res, key=lambda x: (x[0], x[1]))
    res = [{'cx':int(x[0]), 'cy':int(x[1]), 'r':int(x[2])} for x in res]

    preds.append(res)

df['pred'] = preds
df['region_list'] = df.region_shape.apply(lambda x: sorted(json.loads(x.replace("'", "")), key=lambda x: (int(x['cx']), int(x['cy']))))
df[['ID_img', 'region_list', 'pred']]

,ID_img,region_list,pred
8,3444.jpg,"[{'cx': 2259, 'cy': 391, 'r': 64}]","[{'cx': 2248, 'cy': 377, 'r': 60}]"
217,3653.JPG,"[{'cx': 2719, 'cy': 2097, 'r': 75}]","[{'cx': 2714, 'cy': 2083, 'r': 70}]"
254,3690.JPG,"[{'cx': 2914, 'cy': 1693, 'r': 90}]","[{'cx': 2914, 'cy': 1692, 'r': 69}]"
399,3835.JPG,"[{'cx': 2437, 'cy': 2321, 'r': 74}, {'cx': 2549, 'cy': 2329, 'r': 80}]","[{'cx': 2431, 'cy': 2288, 'r': 88}, {'cx': 2562, 'cy': 2310, 'r': 81}]"
511,3947.JPG,"[{'cx': 3130, 'cy': 1370, 'r': 134}]","[{'cx': 3131, 'cy': 1360, 'r': 116}]"
717,4153.jpg,"[{'cx': 3731, 'cy': 3049, 'r': 75}]","[{'cx': 3728, 'cy': 3047, 'r': 54}]"
833,4269.JPG,"[{'cx': 3284, 'cy': 1706, 'r': 81}, {'cx': 3350, 'cy': 1632, 'r': 60}, {'cx': 3475, 'cy': 1586, 'r': 77}, {'cx': 3575, 'cy': 1657, 'r': 98}, {'cx': 4243, 'c...","[{'cx': 3281, 'cy': 1706, 'r': 61}, {'cx': 3349, 'cy': 1636, 'r': 51}, {'cx': 3476, 'cy': 1571, 'r': 61}, {'cx': 3582, 'cy': 1654, 'r': 77}, {'cx': 4241, 'c..."
855,4291.JPG,"[{'cx': 3467, 'cy': 1740, 'r': 77}, {'cx': 3587, 'cy': 1785, 'r': 79}]","[{'cx': 3471, 'cy': 1739, 'r': 66}, {'cx': 3573, 'cy': 1771, 'r': 61}]"
1029,4465.JPG,"[{'cx': 2430, 'cy': 1926, 'r': 107}, {'cx': 2735, 'cy': 1711, 'r': 102}]","[{'cx': 2438, 'cy': 1925, 'r': 86}, {'cx': 2744, 'cy': 1705, 'r': 84}]"
1168,4604.JPG,"[{'cx': 3114, 'cy': 1611, 'r': 133}, {'cx': 3276, 'cy': 1619, 'r': 130}]","[{'cx': 3115, 'cy': 1597, 'r': 85}, {'cx': 3283, 'cy': 1616, 'r': 104}]"


In [4]:
%%time

df = pd.read_csv('sample_solution.csv')

preds = []
for _, row in df.iterrows():
    img = row['ID_img']
    with torch.no_grad():
        results = model(f'test/{img}', augment=True)
    a = results.pred[0].cpu().numpy()
    a = a[a[:, 4] > TH]
    res = list(zip((a[:, 2] + a[:, 0]) / 2, (a[:, 1] + a[:, 3]) / 2, ((a[:, 2] - a[:, 0]) + (a[:, 3] - a[:, 1])) / DIV))
    res = sorted(res, key=lambda x: (x[0], x[1]))
    res = ", ".join([f"'{json.dumps({'cx':int(x[0]), 'cy':int(x[1]), 'r':int(x[2])}).replace(' ', '')}'" for x in res])
    if len(res):
        res = '[' + res + ']'
    else:
        res = '0.0'
    preds.append(res)

df['region_shape'] = preds

print(len(df[df.region_shape != '0.0']))
df[['ID_img', 'region_shape']].to_csv(f'sumb_320v5_{TH}_{DIV}.csv', index=False)
df.head()

375
CPU times: user 3min 7s, sys: 4.43 s, total: 3min 11s
Wall time: 3min 11s


,ID_img,region_shape
0,1.JPG,0.0
1,2.JPG,0.0
2,3.JPG,0.0
3,4.JPG,0.0
4,5.JPG,0.0
